In [2]:
from qiskit import *

In [3]:
tablero = "1000010000010010"

#   0   1   2   3
#   4   5   6   7
#   8   9  10  11
#  12  13  14  15

qc = QuantumCircuit(16 + 6 + 1, 1)

def añadir_tablero(qc, tablero):
    for ind, i in enumerate(tablero):
        if i == "1":
            qc.x(ind)
    return qc
            
qc = añadir_tablero(qc, tablero)

def añadir_cswap(qc, control, columna1, columna2):
    for i in range(len(columna1)):
        qc.cswap(control, columna1[i], columna2[i])
    return qc

def añadir_permutaciones(qc):
    
    qc = añadir_cswap(qc, 16, [0,4,8,12], [1,5,9,13])
    qc = añadir_cswap(qc, 17, [0,4,8,12], [2,6,10,14])
    qc = añadir_cswap(qc, 18, [0,4,8,12], [3,7,11,15])
    qc = añadir_cswap(qc, 19, [1,5,9,13], [2,6,10,14])
    qc = añadir_cswap(qc, 20, [1,5,9,13], [3,7,11,15])
    qc = añadir_cswap(qc, 21, [2,6,10,14], [3,7,11,15])
    return qc

def deshacer_permutaciones(qc):
    
    qc = añadir_cswap(qc, 21, [2,6,10,14], [3,7,11,15])
    qc = añadir_cswap(qc, 20, [1,5,9,13], [3,7,11,15])
    qc = añadir_cswap(qc, 19, [1,5,9,13], [2,6,10,14])
    qc = añadir_cswap(qc, 18, [0,4,8,12], [3,7,11,15])
    qc = añadir_cswap(qc, 17, [0,4,8,12], [2,6,10,14])
    qc = añadir_cswap(qc, 16, [0,4,8,12], [1,5,9,13])
    return qc
    

def añadir_oraculo(qc):
    
    qc.h(12)
    qc.mct([3,6,9], 12)
    qc.h(12)

    return qc


def añadir_grover(qc):
    qc.h(range(16,22))
    qc.x(range(16,22))
    qc.h(21)
    qc.mct(list(range(16,21)),21)
    qc.h(21)
    qc.x(range(16,22))
    qc.h(range(16,22))
    return qc



qc.h(range(16,22))

for _ in range(2):
    
    qc = añadir_permutaciones(qc)
    qc = añadir_oraculo(qc)
    qc = deshacer_permutaciones(qc)
    qc = añadir_grover(qc)

qc = añadir_permutaciones(qc)
qc.mct([3,6,9,12], 22)
qc = deshacer_permutaciones(qc)
qc.measure(22,0)

backend = Aer.get_backend("qasm_simulator")
job = execute(qc, backend, shots = 1000)
job.result().get_counts()
#qc.draw(output = "mpl")

# 1 no tiene solución, 0 tiene solución

{'0': 90, '1': 910}